### Testing Playground for P0 Programs &ndash; Array Indexing and Subarrays

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.
   <br>Alternatively, selecting `Cell > Run All` is sufficient to rerun if there are no code changes.

In [ ]:
# Assigning subarrays
test1 = """
program p
    var a: [1..5] → integer
    var b: [1..3] → integer
    a := [103..107]
    b := a[1:4]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Comparing subarrays
test2 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [6..10]
    b := [6..10]
    if a[1:4] = b[1:4] then write(100)
    if a[1:4] = b[2:5] then write(200)
    if a[1:4][0:2] = b[1:4][0:2] then write(300)
    if a[1:4][0:2] = b[2:5][0:2] then write(400)
"""

# Array variable index below lower bound
test3 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[0])
"""

# Array variable index at lower bound
test4 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[1])
"""

# Array variable index at upper bound
test5 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[3])
"""

# Array variable index above upper bound
test6 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[4])
"""

# Subarray index below lower bound
test7 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][-1])
"""

# Subarray index at lower bound
test8 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][0])
"""

# Subarray index at upper bound
test9 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][2])
"""

# Subarray index above upper bound
test10 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][3])
"""

# Array literal index below lower bound
test11 = """
program p
    write([6..8][-1])
"""

# Array literal index at lower bound
test12 = """
program p
    write([6..8][0])
"""

# Array literal index at upper bound
test13 = """
program p
    write([6..8][2])
"""

# Array literal index above upper bound
test14 = """
program p
    write([6..8][3])
"""

# Array variable subarray covering full range
test15 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[1:4]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Array variable subarray below lower bound
test16 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[0:4]
"""

# Array variable subarray above upper bound
test17 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[1:5]
"""

# Subarray of subarray covering full range
test18 = """
program p
    var a: [1..5] → integer
    var b: [1..3] → integer
    a := [6..10]
    b := a[1:5][0:3]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Subarray of subarray below lower bound
test19 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [6..10]
    b := a[1:5][-1:3]
"""

# Subarray of subarray above upper bound
test20 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [6..10]
    b := a[1:5][0:4]
"""

# Array literal subarray covering full range
test21 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][0:3]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Array literal subarray below lower bound
test22 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][-1:3]
"""

# Array literal subarray above upper bound
test23 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][0:4]
"""

# Subarray with non-increasing range
test24 = """
program p
    var a: [1..3] → integer
    a := [6..8][2:1]
"""

# Update this line to run a specific testcase
compileString(test1, "test.wat")

In [ ]:
!cat -n "test.wat"

In [ ]:
!wat2wasm --enable-bulk-memory test.wat || rm test.wasm

In [ ]:
runwasm("test.wasm")

In [ ]:
time.sleep(1)

The above tests can also be set to run all at once using the following steps:
1. Define testcases using the following format: `(testcase, pass, result)`
   <br>`testcase` is a string such as the above containing a P0 program to be tested.
   <br>`pass` is a Boolean value set to `True` if the program should compile/run without raising an exception.
   <br>`result` is the expected output or exception message when the given program is compiled/run.
2. Set the value of `runall` to `True` below (set to `False` by default to prevent unnecessary runs).
3. Rerun the notebook as above, or run the below cells by selecting `Cell > Run All Below` from the below cell.

Note: the automated test runs should ideally be modified below to capture the output of the `runwasm` function and compare
<br>against `test[2]` to check for the expected result if the program runs successfully, but this has not yet been implemented.

In [ ]:
tests = [
    (test1, True, "103\n104\n105"),
    (test2, True, "100\n300"),
    (test3, False, "index out of bounds"),
    (test4, True, "6"),
    (test5, True, "8"),
    (test6, False, "index out of bounds"),
    (test7, False, "index out of bounds"),
    (test8, True, "6"),
    (test9, True, "8"),
    (test10, False, "index out of bounds"),
    (test11, False, "index out of bounds"),
    (test12, True, "6"),
    (test13, True, "8"),
    (test14, False, "index out of bounds"),
    (test15, True, "6\n7\n8"),
    (test16, False, "start index out of bounds"),
    (test17, False, "end index out of bounds"),
    (test18, True, "6\n7\n8"),
    (test19, False, "start index out of bounds"),
    (test20, False, "end index out of bounds"),
    (test21, True, "6\n7\n8"),
    (test22, False, "start index out of bounds"),
    (test23, False, "end index out of bounds"),
    (test24, False, "range must be increasing"),
]

In [ ]:
%%capture errors
runall = False
passing = 0
if runall:
    for index, test in enumerate(tests):
        try:
            compileString(test[0], "test.wat")
            !wat2wasm --enable-bulk-memory test.wat || rm test.wasm
            runwasm("test.wasm")
            # TODO: compare runwasm output to test[2]
            if test[1]: passing += 1
            else: print(f'Test #{index + 1} did not throw an exception')
        except Exception as e:
            if not test[1] and test[2] in str(e): passing += 1
            elif not test[1]: print(f'Test #{index + 1} threw an unexpected exception: {e}')
            else: print(f'Test #{index + 1} threw an unhandled exception: {e}')

In [ ]:
if runall:
    print(f'{passing} out of {len(tests)} tests pass')
    print(errors)
else:
    print("Skipped automated testing")